In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

# Data Collection

## Downloading Preprocessed Data

In [ ]:
!wget "https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/Datasets/House_Prices_AdvReg_preprocessing.zip"

--2021-09-16 12:39:05--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/Datasets/House_Prices_AdvReg_preprocessing.zip
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)... 52.219.160.42
Connecting to nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)|52.219.160.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 899241 (878K) [application/zip]
Saving to: ‘House_Prices_AdvReg_preprocessing.zip’

House_Prices_AdvReg 100%[===================>] 878.17K  1.73MB/s    in 0.5s    

2021-09-16 12:39:06 (1.73 MB/s) - ‘House_Prices_AdvReg_preprocessing.zip’ saved [899241/899241]



In [ ]:
!unzip "House_Prices_AdvReg_preprocessing.zip"

Archive:  House_Prices_AdvReg_preprocessing.zip
  inflating: test_X_preprocessed.csv  
  inflating: train_preprocessed.csv  


In [ ]:
train_df = pd.read_csv('train_preprocessed.csv')
test_X_df = pd.read_csv('test_X_preprocessed.csv')
train_df

,Id,MSSubClass,LotFrontage,LotArea,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,...,MSZoning_RL,MSZoning_RM,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Alley_Grvl,Alley_Pave,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,SalePrice
0,1,0.073350,-0.223281,-0.207071,0.225639,0.651256,-0.517023,1.050634,0.878367,0.511288,1.051942,-0.238031,0.582968,0.117844,-0.317784,1.164313,0.575228,-0.277094,-0.288554,-0.944267,-0.459145,0.890874,0.274055,-0.793162,1.161454,-0.120201,0.370207,1.107431,-0.240978,0.789470,1.227165,0.163723,-0.211381,0.735742,0.911897,0.236962,-0.950901,-1.007988,1.019654,-1.001340,...,0.517956,-0.418812,-0.703962,-0.169923,-0.083017,0.760251,-0.026171,0.149645,-0.111688,-0.069385,-0.037024,-0.052396,-0.174141,-0.052396,-0.037024,-0.07873,-0.058601,-0.058601,-0.301858,-0.045361,0.390159,0.169923,-0.169923,-0.343273,-0.098363,-0.994195,-0.074202,-0.087099,1.509747,-0.161194,-0.215785,-0.026171,0.134606,-0.026171,-0.026171,-0.026171,-0.087099,-0.058601,-0.064216,208500
1,2,-0.872264,0.457737,-0.091855,0.225639,-0.071812,2.178881,0.156680,-0.429430,-0.574169,-0.689368,-0.238031,0.582968,0.117844,3.144637,0.689879,1.171591,-0.277094,-0.288554,-0.641008,0.466305,0.890874,0.274055,0.257052,-0.794891,-0.120201,-0.482347,-0.819684,3.947457,0.789470,-0.761360,0.163723,-0.211381,-0.770827,-0.318574,0.236962,0.600289,0.648668,-0.105588,-1.001340,...,0.517956,-0.418812,-0.703962,-0.169923,-0.083017,0.760251,-0.026171,0.149645,-0.111688,-0.069385,-0.037024,-0.052396,-0.174141,-0.052396,-0.037024,-0.07873,-0.058601,-0.058601,-0.301858,-0.045361,0.390159,0.169923,-0.169923,-0.343273,-0.098363,1.005150,-0.074202,-0.087099,-0.661909,-0.161194,-0.215785,-0.026171,0.134606,-0.026171,-0.026171,-0.026171,-0.087099,-0.058601,-0.064216,181500
2,3,0.073350,-0.087077,0.073455,0.225639,0.651256,-0.517023,0.984415,0.829930,0.322995,1.051942,-0.238031,0.582968,0.117844,-0.317784,1.164313,0.092875,-0.277094,-0.288554,-0.301540,-0.313261,0.890874,0.274055,-0.627611,1.188943,-0.120201,0.514836,1.107431,-0.240978,0.789470,1.227165,0.163723,-0.211381,0.735742,-0.318574,0.236962,0.600289,0.648668,0.936303,-1.001340,...,0.517956,-0.418812,1.419559,-0.169923,-0.083017,-1.314453,-0.026171,0.149645,-0.111688,-0.069385,-0.037024,-0.052396,-0.174141,-0.052396,-0.037024,-0.07873,-0.058601,-0.058601,-0.301858,-0.045361,0.390159,0.169923,-0.169923,-0.343273,-0.098363,-0.994195,-0.074202,-0.087099,1.509747,-0.161194,-0.215785,-0.026171,0.134606,-0.026171,-0.026171,-0.026171,-0.087099,-0.058601,-0.064216,223500
3,4,0.309753,-0.450286,-0.096864,0.225639,0.651256,-0.517023,-1.862993,-0.720051,-0.574169,-0.689368,-0.238031,-0.557961,1.928918,-0.317784,0.689879,-0.499103,-0.277094,-0.288554,-0.061648,-0.687089,-0.151334,0.274055,-0.521555,0.936955,-0.120201,0.383528,1.107431,-0.240978,-1.025689,-0.761360,0.163723,-0.211381,0.735742,0.296662,0.236962,0.600289,1.200886,0.811276,-0.121174,...,0.517956,-0.418812,1.419559,-0.169923,-0.083017,-1.314453,-0.026171,0.149645,-0.111688,-0.069385,-0.037024,-0.052396,-0.174141,-0.052396,-0.037024,-0.07873,-0.058601,-0.058601,-0.301858,-0.045361,0.390159,0.169923,-0.169923,-0.343273,-0.098363,-0.994195,-0.074202,-0.087099,1.509747,-0.161194,-0.21

Separating SalePrice as Target Variable

In [ ]:
train_Y_df = train_df.pop('SalePrice')
train_X_df = train_df

Converting to numpy arrays

In [ ]:
#removing Id column as it's not required in training
train_X = train_X_df.to_numpy()[:,1:] 
test_X = test_X_df.to_numpy()[:,1:] 

train_Y = train_Y_df.to_numpy()

# Ridge Regression Algorithm

## Using Gradient Descent

### Cost Function

In [ ]:
def compute_cost_ridge_regression(X, Y, theta, Lambda):
  H = np.dot(X, theta)
  difference = H - Y
  m = len(X)
  n = len(theta)
  I = np.eye(n, dtype=int)
  I[0][0] = 0
  regularization_term = (Lambda/(2*m))*np.dot(np.dot(theta.T, I), theta)
  J = (1/(2*m))*np.dot(difference.T, difference) + regularization_term
  return(np.squeeze(J))

### Gradient Descent

In [ ]:
def gradient_of_cost_function(X, Y, theta, Lambda):
    H = np.dot(X, theta)
    difference =  H - Y
    m = len(X)
    n = len(theta)
    I = np.eye(n, dtype=int)
    I[0][0] = 0
    regularization_term = Lambda*np.dot(I, theta)
    d_theta = (1/m)*(np.dot(X.T,difference) + regularization_term)
    return d_theta

def gradient_descent_ridge_regression(X, Y, theta, Lambda, cost_diff_threshold, learning_rate):
    i = 0
    cost_diff = cost_diff_threshold + 1
    thetas = [theta]
    costs = [compute_cost_ridge_regression(X, Y, theta, Lambda)]

    while(abs(cost_diff) > cost_diff_threshold):
        d_theta = gradient_of_cost_function(X, Y, theta, Lambda)
        theta = theta - (learning_rate * d_theta)
        costs.append(compute_cost_ridge_regression(X, Y, theta, Lambda))
        thetas.append(theta)
        cost_diff = costs[i+1] - costs[i]
        if(cost_diff > 0): #Diverging case
            print("Diverging")
            break
        i = i+1
    return thetas, costs

## Normal Equation Method

In [ ]:
def normal_equation_ridge_regression(X, Y, Lambda):
  XT_X = np.dot(X.T, X)
  n = len(X[0])
  I = np.eye(n, dtype=int)
  I[0][0] = 0
  inverse = np.linalg.pinv(XT_X + Lambda*I) 
  XT_Y = np.dot(X.T, Y)
  result = np.dot(inverse,XT_Y)
  return result

## Evaluation using MSE

In [ ]:
def MSE(predicted_Y, actual_Y):
    difference = actual_Y - predicted_Y
    m = len(predicted_Y)
    mse = (1/m)*np.dot(difference.T, difference)
    return np.squeeze(mse)

# Applying Regression Utils

In [ ]:
np.random.seed(11235811)
data_count = train_X.shape[0]
permuted_indices = np.random.permutation(data_count)

train_ratio = 0.8
train_count = np.int(train_ratio * data_count)

## Splitting into Train, Validation & Test sets

In [ ]:
def split_train_val(X, Y):
    shuffled_X = X[permuted_indices]
    shuffled_Y = Y[permuted_indices]

    train_X = shuffled_X[:train_count]
    train_Y = shuffled_Y[:train_count]

    val_X = shuffled_X[train_count:]
    val_Y = shuffled_Y[train_count:]
    
    return train_X, train_Y, val_X, val_Y

## Using Gradient Descent

#### Preparing initial $\theta$

In [ ]:
def get_initial_theta(num_parameters):
    theta = np.random.rand(num_parameters,1)
    return theta

#### Train using GD

In [ ]:
def apply_GD(train_X, train_Y, Lambda): 
    theta = get_initial_theta(train_X.shape[1])  
    cost_diff_threshold = 1e-6
    learning_rate = 0.12
    thetas, costs = gradient_descent_ridge_regression(train_X, train_Y, theta, Lambda, cost_diff_threshold, learning_rate)
    theta_GD = thetas[-1]
    return theta_GD, costs

#### Prediction & Evaluation

In [ ]:
def evaluate(data_X, data_Y, theta):
    predicted_data_Y = np.dot(data_X, theta)
    mse = MSE(predicted_data_Y, data_Y)
    return mse, predicted_data_Y

# Applying Ridge Regression

Linear Regression with $L_2$ regularization (Ridge regression)

In [ ]:
def ridge_regression(train_X, train_Y, Lambda, split_data=True, use_Normal_Eqn=True):
    train_X = np.insert(train_X, 0, 1, axis=1)
    train_Y = train_Y.reshape((-1,1))

    if(split_data):
        train_X, train_Y, val_X, val_Y = split_train_val(train_X, train_Y)

    if(use_Normal_Eqn):
        theta = normal_equation_ridge_regression(train_X, train_Y, Lambda)
    else:
        theta, costs = apply_GD(train_X, train_Y, Lambda)
  
    #EVALUATION USING VALIDATION SET
    if(split_data):
        val_mse, predicted_val_Y = evaluate(val_X, val_Y, theta)
        return val_mse, theta
    else:
        return theta

## Tuning hyperparameter $\lambda$

* If `use_Normal_Eqn=False`, then gradient descent is used in training. 
* If `use_Normal_Eqn=True`, then the Normal Equation Method is used in training.

In [ ]:
Lambda_values = np.array([0, 1, 10, 50, 100, 113, 150])
val_MSEs = []
for Lambda in Lambda_values:
    val_mse, theta = ridge_regression(train_X, train_Y, Lambda, split_data=True)
    val_MSEs.append(val_mse)
    print(Lambda, val_mse)

min_val_index = np.argmin(val_MSEs)
optimal_Lambda = Lambda_values[min_val_index]
print("optimal λ:", optimal_Lambda, "\nminimum validation error:", val_MSEs[min_val_index])

0 1084759199.7975698
1 632198444.9241636
10 618509936.3354946
50 596835972.2537178
100 591558286.8515968
113 591552518.3799363
150 593136251.0798382
optimal λ: 113 
minimum validation error: 591552518.3799363


We can observe that regularization increases the performance of the model<br> by looking at the difference between the validation MSEs:
- with $\lambda=0$ (NO regularization), 
- with $\lambda=\text{optimal_Lambda}$ (with regularization)

In [ ]:
print(val_MSEs[0] - val_MSEs[min_val_index])

493206681.4176334


## Training using the entire training data (train set + val set)

In [ ]:
Lambda = optimal_Lambda
theta = ridge_regression(train_X, train_Y, Lambda, split_data=False)

## Predicting for test data

In [ ]:
test_X = np.insert(test_X, 0, 1, axis=1)
predicted_test_Y = np.dot(test_X, theta)

In [ ]:
predicted_test_Y

array([[117651.33962912],
       [172108.48183392],
       [178123.51455506],
       ...,
       [174423.71765367],
       [104825.0054942 ],
       [220319.1735336 ]])